In [16]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import logging
import json

# Logging 설정
logging.basicConfig(level='INFO', format='%(asctime)s %(name)s: %(message)s', datefmt='%Y-%m-%dT%H:%M:%S')
logger = logging.getLogger(__name__)

In [17]:
def get_model():
    try:
        model = ChatGoogleGenerativeAI(
            google_api_key=os.getenv("GEMINI_API_KEY"),
            model="gemini-2.5-flash",
            temperature=0,
            max_tokens=4048
        )
        logger.info("llm 모델이 성공적으로 초기화되었습니다.")
        return model
    except Exception as e:
        logger.error(f"모델 초기화 중 오류 발생 : {e}")
        raise e

In [18]:
def extract_entities(raw_sentence):
        model = get_model()
        prompt =f"""
        You are an expert at extracting key entities from the sentence given.
        The sentence is about book request information and our aim is to identify the main entities in the sentence along with their importance weight.
        The weight should range from 0 to 1, where 1 indicates the highest importance.
        # Sentence: "{raw_sentence}"
        # Output Format :
        {{
            "target_age":{{"value": "age_value1", "weight": weight_value1}}, 
            "book_category":{{"value": "category_value", "weight": weight_value2}},
            "book_amount": {{"value": "category_value", "weight": weight_value2}},
            "others": {{"entity1": {{"value": "value1", "weight": weight1}}, "entity2": {{"value": "value2", "weight": weight2}}...}}
            
        }}

        # **Instructions** :
        Include all the other entities that are not related to age, category, and amount to 'others'.
        When there are no entities for age, category or amount, set their value to null and weight to 0.
        If the sentence is not about book requests, return the following JSON: {{"message": "The sentence does not contain book request information."}}.
        Please provide the output in pure JSON format only.
        """

        response = model.invoke(prompt)

        try:
            parsed = json.loads(response.content)
            logger.info("LLM 개체 추출 성공")
            return parsed
        except json.JSONDecodeError as e:
            logger.warning(f"JSON 구문 분석 오류: {e}")
            return response

In [19]:
test_sentence = "안녕하세요, 저는 10대 청소년들을 위한 판타지 소설을 5권 정도 구입하고 싶습니다."
result = extract_entities(test_sentence)
print(result)

2025-11-14T16:06:22 __main__: 모델 초기화 중 오류 발생 : Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.